In [1]:
#This version of the code is simpler, just pulling regional forecast data over the period without the generation mix

import requests
import csv
import os
from datetime import datetime, timedelta

# === CONFIGURATION ===
start_datetime_str = "2024-06-30T23:00Z"
end_datetime_str   = "2025-06-30T23:00Z"

output_dir = r"C:\Users\spice\Dropbox\Documents\Imperial 2024.2025\MECH70038 - Research Projects\_My Thesis\Data"
os.makedirs(output_dir, exist_ok=True)
output_filename = f"regional_forecast_matrix_named_{start_datetime_str[:10]}_to_{end_datetime_str[:10]}.csv"
output_path = os.path.join(output_dir, output_filename)

headers = {'Accept': 'application/json'}

# === PARSE DATES ===
def parse_utc(datestr):
    return datetime.strptime(datestr, "%Y-%m-%dT%H:%MZ")

start_dt = parse_utc(start_datetime_str)
end_dt   = parse_utc(end_datetime_str)
max_window = timedelta(days=13, hours=23, minutes=30)

# === INITIALISE STRUCTURES ===
forecast_data = {}  # key: (from, dnoregion), value: forecast
all_regions = set()

# === LOOP OVER API CHUNKS ===
curr_start = start_dt
while curr_start < end_dt:
    curr_end = min(curr_start + max_window, end_dt)
    from_str = curr_start.strftime("%Y-%m-%dT%H:%MZ")
    to_str   = curr_end.strftime("%Y-%m-%dT%H:%MZ")

    print(f"Fetching {from_str} to {to_str}...")
    url = f"https://api.carbonintensity.org.uk/regional/intensity/{from_str}/{to_str}"
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    timeblocks = response.json()["data"]

    for block in timeblocks:
        timestamp = block["from"]
        for region in block.get("regions", []):
            region_name = region.get("dnoregion")
            all_regions.add(region_name)
            forecast = region.get("intensity", {}).get("forecast")
            forecast_data[(timestamp, region_name)] = forecast

    curr_start = curr_end

# === GENERATE FULL TIME RANGE ===
full_times = []
dt = start_dt
while dt < end_dt:
    from_str = dt.strftime("%Y-%m-%dT%H:%MZ")
    to_str   = (dt + timedelta(minutes=30)).strftime("%Y-%m-%dT%H:%MZ")
    full_times.append((from_str, to_str))
    dt += timedelta(minutes=30)

# === BUILD FINAL ROWS ===
sorted_regions = sorted(all_regions)
fieldnames = ["from", "to"] + sorted_regions
all_rows = []

for from_str, to_str in full_times:
    row = {"from": from_str, "to": to_str}
    for region in sorted_regions:
        row[region] = forecast_data.get((from_str, region), "")
    all_rows.append(row)

# === WRITE TO CSV ===
with open(output_path, mode="w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(all_rows)

print(f"Saved named-region forecast matrix to: {output_path}")


Fetching 2024-06-30T23:00Z to 2024-07-14T22:30Z...
Fetching 2024-07-14T22:30Z to 2024-07-28T22:00Z...
Fetching 2024-07-28T22:00Z to 2024-08-11T21:30Z...
Fetching 2024-08-11T21:30Z to 2024-08-25T21:00Z...
Fetching 2024-08-25T21:00Z to 2024-09-08T20:30Z...
Fetching 2024-09-08T20:30Z to 2024-09-22T20:00Z...
Fetching 2024-09-22T20:00Z to 2024-10-06T19:30Z...
Fetching 2024-10-06T19:30Z to 2024-10-20T19:00Z...
Fetching 2024-10-20T19:00Z to 2024-11-03T18:30Z...
Fetching 2024-11-03T18:30Z to 2024-11-17T18:00Z...
Fetching 2024-11-17T18:00Z to 2024-12-01T17:30Z...
Fetching 2024-12-01T17:30Z to 2024-12-15T17:00Z...
Fetching 2024-12-15T17:00Z to 2024-12-29T16:30Z...
Fetching 2024-12-29T16:30Z to 2025-01-12T16:00Z...
Fetching 2025-01-12T16:00Z to 2025-01-26T15:30Z...
Fetching 2025-01-26T15:30Z to 2025-02-09T15:00Z...
Fetching 2025-02-09T15:00Z to 2025-02-23T14:30Z...
Fetching 2025-02-23T14:30Z to 2025-03-09T14:00Z...
Fetching 2025-03-09T14:00Z to 2025-03-23T13:30Z...
Fetching 2025-03-23T13:30Z to 2

In [ ]:
#Time given is UTC. 
#This pulls the regional forecast emission factors and generation. Intensity has a 14 day limit. So batches over longer periods.
#Sometimes model data unavailable - this code keeps all settlement periods and just adds blank rows if data unavailable
import requests
import csv
import os
from datetime import datetime, timedelta

# === CONFIGURATION ===
start_datetime_str = "2025-01-01T00:00Z"
end_datetime_str   = "2025-01-16T00:00Z"

output_dir = r"C:\Users\spice\Dropbox\Documents\Imperial 2024.2025\MECH70038 - Research Projects\_My Thesis\Data"
os.makedirs(output_dir, exist_ok=True)
output_filename = f"regional_forecast_{start_datetime_str[:10]}_to_{end_datetime_str[:10]}.csv"
output_path = os.path.join(output_dir, output_filename)

headers = {'Accept': 'application/json'}

# === PARSE DATES ===
def parse_utc(datestr):
    return datetime.strptime(datestr, "%Y-%m-%dT%H:%MZ")

start_dt = parse_utc(start_datetime_str)
end_dt   = parse_utc(end_datetime_str)
max_window = timedelta(days=13, hours=23, minutes=30)

# === INITIALISE STRUCTURE ===
raw_data = {}  # key: (from, regionid), value: row dict
all_fuels = set()
all_regionids = set()

# === LOOP OVER CHUNKS ===
curr_start = start_dt
while curr_start < end_dt:
    curr_end = min(curr_start + max_window, end_dt)
    from_str = curr_start.strftime("%Y-%m-%dT%H:%MZ")
    to_str   = curr_end.strftime("%Y-%m-%dT%H:%MZ")

    print(f"Fetching {from_str} to {to_str}...")
    url = f"https://api.carbonintensity.org.uk/regional/intensity/{from_str}/{to_str}"
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    timeblocks = response.json()["data"]

    for block in timeblocks:
        timestamp = block["from"]
        timestamp_to = block["to"]
        for region in block.get("regions", []):
            regionid = region["regionid"]
            all_regionids.add(regionid)

            row = {
                "from": timestamp,
                "to": timestamp_to,
                "regionid": regionid,
                "dnoregion": region.get("dnoregion"),
                "forecast": region.get("intensity", {}).get("forecast"),
                "index": region.get("intensity", {}).get("index")
            }

            for fuel_data in region.get("generationmix", []):
                fuel = fuel_data["fuel"]
                row[fuel] = fuel_data["perc"]
                all_fuels.add(fuel)

            raw_data[(timestamp, regionid)] = row

    curr_start = curr_end

# === GENERATE FULL TIME RANGE ===
full_times = []
dt = start_dt
while dt < end_dt:
    from_str = dt.strftime("%Y-%m-%dT%H:%MZ")
    to_str   = (dt + timedelta(minutes=30)).strftime("%Y-%m-%dT%H:%MZ")
    full_times.append((from_str, to_str))
    dt += timedelta(minutes=30)

# === DETERMINE FINAL COLUMN ORDER ===
sorted_fuels = sorted(all_fuels)
fieldnames = ["from", "to", "regionid", "dnoregion", "forecast", "index"] + sorted_fuels

# === BUILD FINAL ROWS ===
all_rows = []
for from_str, to_str in full_times:
    for regionid in sorted(all_regionids):
        key = (from_str, regionid)
        row = raw_data.get(key, {
            "from": from_str,
            "to": to_str,
            "regionid": regionid,
            "dnoregion": "",
            "forecast": "",
            "index": ""
        })
        for fuel in sorted_fuels:
            row.setdefault(fuel, "")
        all_rows.append(row)

# === WRITE TO CSV ===
with open(output_path, mode="w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in all_rows:
        writer.writerow(row)

print(f"Saved combined CSV with filled gaps to: {output_path}")


Fetching 2025-01-01T00:00Z to 2025-01-14T23:30Z...
Fetching 2025-01-14T23:30Z to 2025-01-16T00:00Z...
Saved combined CSV to: C:\Users\spice\Dropbox\Documents\Imperial 2024.2025\MECH70038 - Research Projects\_My Thesis\Data\regional_forecast_2025-01-01_to_2025-01-16.csv
